In [19]:
import os
import pandas as pd
import numpy as np
import random

In [20]:
# Importing the csv lists and merge together

date = 240826
k = 10
d = 5
name_first = f"{d}-1"
name_all = f"{d}-all"
file_path_first = f"C:\\Users\\david\\OneDrive\\Documents\\01_Architektur\\02_MA Architektur\\TUM\\2024_Sommersemester\\01_THESIS\\02_MA_Thesis\\02_Listing\\01_Visual_Studio\\240816_Simulation\\02-1_Clustering-Colour\\{date}_{name_first}_Brick_colour-cluster_k{k}.csv"
file_path_all = f"C:\\Users\\david\\OneDrive\\Documents\\01_Architektur\\02_MA Architektur\\TUM\\2024_Sommersemester\\01_THESIS\\02_MA_Thesis\\02_Listing\\01_Visual_Studio\\240816_Simulation\\02-1_Clustering-Colour\\{date}_{name_all}_Brick_colour-cluster_k{k}.csv"


bd_first = pd.read_csv(file_path_first)

if os.path.exists(file_path_all):
    bd_all = pd.read_csv(file_path_all)
    bd_combined = pd.concat([bd_first, bd_all], ignore_index=True)
else:
    bd_combined = bd_first

#print(brick_df_first.head(5))
#print(brick_df_all.head(5))
print(bd_combined.head(5))
print("")
print(f" There are a total of {len(bd_combined)} bricks in delivery {d}")



   Unnamed: 0  brick_id  delivery_nr  brick_format  \
0        6777      6778            5  Reichsformat   
1        6778      6779            5  Reichsformat   
2        6779      6780            5  Reichsformat   
3        6780      6781            5  Reichsformat   
4        6781      6782            5  Reichsformat   

  brick_original_dimensions (mm) brick_origin  brick_distance (km)  age (y)  \
0                   250, 120, 65      Kempten                  123      117   
1                   250, 120, 65      Kempten                  123      117   
2                   250, 120, 65      Kempten                  123      117   
3                   250, 120, 65      Kempten                  123      117   
4                   250, 120, 65      Kempten                  123      117   

  colour name  turn_nr  mortar rests (%)  colour rests (%)    R   G   B  \
0   red-brown        1              0.35              0.01  165  52  41   
1   red-brown        1              0.16          

In [21]:
def calculate_luminance(R, G, B):
    return 0.2126 * R + 0.7152 * G + 0.0722 * B

def calculate_cov(values):
    mean_val = np.mean(values)
    std_dev = np.std(values)
    return (std_dev / mean_val) * 100 if mean_val != 0 else 0

def process_cluster_data(bd_cluster, cluster_num, date):
    n = len(bd_cluster)
    batch_size = 50
    num_batches = n // batch_size
    remainder = n % batch_size

    batches = []
    
    for i in range(num_batches):
        bd_batch = bd_cluster.iloc[i*batch_size:(i+1)*batch_size].copy()

        Rs = bd_batch['R'].values
        Gs = bd_batch['G'].values
        Bs = bd_batch['B'].values

        luminances = [calculate_luminance(R, G, B) for R, G, B in zip(Rs, Gs, Bs)]
        luminances = np.array(luminances)
        avg_luminance = np.mean(luminances)

        #print(f"Cluster {cluster_num}, Batch {i + 1}:")
        #print(f"Average Luminance: {avg_luminance:.2f}")

        batches.append((bd_batch, avg_luminance))

   
    if remainder > 0: 
        bd_remainder = bd_cluster.iloc[num_batches*batch_size:].copy()

        Rs = bd_remainder['R'].values
        Gs = bd_remainder['G'].values
        Bs = bd_remainder['B'].values

        luminances = [calculate_luminance(R, G, B) for R, G, B in zip(Rs, Gs, Bs)]
        luminances = np.array(luminances)
        avg_luminance = np.mean(luminances)

        #print(f"Cluster {cluster_num}")
        #print(f"Average Luminance: {avg_luminance:.2f}")

        batches.append((bd_remainder, avg_luminance))


    # Change sorting to descending order based on average luminance
    batches.sort(key=lambda x: x[1], reverse=True)

    
    overall_avg_luminance = np.mean([batch[1] for batch in batches])

    return (batches, overall_avg_luminance)

def save_sorted_batches(sorted_clusters, date):
    brightness_rank = 1
    print("\nExport Order:")
    for cluster_num, batches, _ in sorted_clusters:
        print(f"\nCluster {cluster_num}:")
        for idx, (batch_df, avg_luminance) in enumerate(batches, start=1):
            output_file = f"{date}_{d}_{brightness_rank}_k-{cluster_num}-{k}_{idx}.csv"
            batch_df.to_csv(output_file, index=False)
            
            
            avg_R = batch_df['R'].mean()
            avg_G = batch_df['G'].mean()
            avg_B = batch_df['B'].mean()
            
            print(f"  Batch {idx}: RGB({avg_R:.2f}, {avg_G:.2f}, {avg_B:.2f}), Luminance: {avg_luminance:.2f}, File: {output_file}")
            
        brightness_rank += 1

clusters = bd_combined["colour_clusters"].unique()

all_clusters = []

for cluster in clusters:
    cluster_df = bd_combined[bd_combined["colour_clusters"] == cluster]
    batches, overall_avg_luminance = process_cluster_data(cluster_df, cluster, date)
    all_clusters.append((cluster, batches, overall_avg_luminance))

# Sort clusters by overall average luminance in descending order
all_clusters.sort(key=lambda x: x[2], reverse=True)

# Save the batches, with clusters sorted by darkness
save_sorted_batches(all_clusters, date)


Export Order:

Cluster 2:
  Batch 1: RGB(229.00, 153.00, 68.00), Luminance: 163.02, File: 240826_5_1_k-2-10_1.csv

Cluster 0:
  Batch 1: RGB(143.00, 92.00, 22.00), Luminance: 97.79, File: 240826_5_2_k-0-10_1.csv

Cluster 9:
  Batch 1: RGB(194.00, 69.00, 34.00), Luminance: 93.05, File: 240826_5_3_k-9-10_1.csv

Cluster 5:
  Batch 1: RGB(137.00, 79.00, 35.00), Luminance: 88.15, File: 240826_5_4_k-5-10_1.csv

Cluster 7:
  Batch 1: RGB(229.00, 50.00, 47.00), Luminance: 87.84, File: 240826_5_5_k-7-10_1.csv

Cluster 4:
  Batch 1: RGB(178.35, 54.57, 42.03), Luminance: 79.98, File: 240826_5_6_k-4-10_1.csv

Cluster 8:
  Batch 1: RGB(123.00, 72.00, 27.00), Luminance: 79.59, File: 240826_5_7_k-8-10_1.csv

Cluster 1:
  Batch 1: RGB(186.00, 38.00, 32.00), Luminance: 69.03, File: 240826_5_8_k-1-10_1.csv

Cluster 6:
  Batch 1: RGB(141.00, 45.00, 28.00), Luminance: 64.18, File: 240826_5_9_k-6-10_1.csv

Cluster 3:
  Batch 1: RGB(88.97, 19.00, 15.34), Luminance: 33.61, File: 240826_5_10_k-3-10_1.csv
